# Water Pipe Length Calculation Tool

As Input the tool will ask for the population, the area and the country or region that is to be investigated.
The input for "country/region" is limited, as it will later be used draw on the following table containing data for the percentage of population of a country using basic water services. The table was downloaded from http://hdr.undp.org/en/data under the dimension "Socio-economic-sustainability" and selecting "Population using at least basic drinking water services (%)".

In [1]:
pop_services <- read.csv("Population_water_services.csv", header=TRUE, sep=",", dec=".", na.strings="..")

There is no data available for Hong Kong, Liechtenstein and Palastine. For Hong Kong and Liechtenstein 100% access is assumed due to the high human development index (HDI) value. For Palastine, which has a much lower HDI, the value for the category "countries with low human development" is assumed
For Argentina, Central African Republic, Dominica, Estonia, Saint Kittis and Nevis, there are no values for 2017, so the values for 2016 are used, except for Dominica and Saint Kittis and Nevis. For these two states 2016 data is also unavailable, so the most recent available datapoint is used, namely the 2015 value for Dominica and the 2013 value for Saint Kitts and Nevis.
The table contains values for the years 2000-2017. For this tool only the most recent data is required.
For convenience the countries/regions are stored in a separate vector and then entered into a matrix for better readibility.

In [2]:
pop_water <- pop_services[,c(1,2,37)]
countries <- pop_water[,2]
regions <- matrix(countries, byrow=FALSE, ncol=5)

If not a whole country but only an urban region is analyzed, it is assumed that the percentage of people using basic drinking water services is higher than for the entire country. Therefore, for all countries/regions with an original value lower than 90%, an extra 10% is added. There is no differentiation between regions for lack of data. The information is stored in an extra dataframe and is being called upon in the function depending on the population density.

In [3]:
pop_water_ur <- pop_water[,3]
for(i in 1:length(pop_water_ur)) {
  if (pop_water_ur[i]<=90){
    pop_water_ur[i]=pop_water_ur[i]+10 
  } else {
    pop_water_ur[i]=pop_water_ur[i]
  }}
pop_water_urban <- data.frame(pop_water[,2], pop_water_ur)

For the next step researched population density and pipe length per citizen (in km) data of five cities/regions is recorded. The reviewed sample regions are (from small to large): Eschenburg, Siegen, Berlin (all in Germany), Santiago de Chile and New York City.

In [4]:
dens <- c(228, 897, 4088, 8407, 10640)
len <- c(0.0116, 0.0084, 0.0027, 0.0015, 0.00129)
rel_dens_len <- data.frame(dens, len)

A cubic function is fitted to the datapoints and the calculated factors stored in variables.

In [5]:
facfun <- lm(len~dens+I(dens^2)+I(dens^3), data=rel_dens_len)
a <- as.numeric(facfun$coefficients[1])
b <- as.numeric(facfun$coefficients[2])
c <- as.numeric(facfun$coefficients[3])
d <- as.numeric(facfun$coefficients[4])

Below a function is build using the arguments country/region (x), population (y) and population density (z) which will be asked to be inserted by the user or respectively calculated by using the inputs:
1.) For the base length the population of the region in question is multiplied with a funciton which represents the relationship of the population density and the length of pipe per citizen as calculated by researched datapoints (see above).
2.) Then a correction factor for the percentage of people having access to basic water supply is applied. It is based on the UN Index of people using at least basic drinking water services. This is not limited to piped water, however for lack of a better estimate, it is assumed that it comes close. If a solely urban area is investigated, the factor is corrected by an increase of 10% as described above. An area is considered primarely urban if the population density calculated from the inputs surpasses 680 people per km². This number is based on a comprehensive review of population density data for countries. Most countries, excluding city states, count with a population density below 680 people per km², so it can be reasonably assumed that an area surpassing this value is either a city state or an urban region.

In [6]:
pipe_length <- function(x, y, z){
#1.)
    base_length <- y * (a + z * b
                               + c * z^2 + d * z^3)
#2.)
  if (z>=680){
    access <- pop_water_urban[grep(x, pop_water_urban[,1]),2]/100
  } else {
    access <- pop_water[grep(x, pop_water[,2]),3]/100
  }
  final_length <- base_length * access
  return(final_length)
}


Subsequent the final tool is designed. First the valid inputs for the country/region field are listed. Then the user is prompted to insert country/region, population and area. These values are stored in variables. Population and area are used to calculate the population density. The density, in turn, along with the country/region and the population count is then subjected to the function determining the pipe length which is defined above. Finally, the function below returns the estimate including its unit.

In [7]:
input <- function(){
  print("Valid Inputs: ", quote=FALSE)
  print(regions)
  country <- readline (prompt="Enter country or region: ")
  population <- as.numeric(readline (prompt="Enter population: "))
  area <- as.numeric(readline(prompt="Enter area in km²: "))
  pop_density <- population/area
  final <- pipe_length(country, population, pop_density)
  paste0(final, " km")
}

The final line of code calls on the input function defined above and let's the user execute the whole code.

In [23]:
input()

Enter country or region: Germany
Enter population: 10500
Enter area in km²: 46
[1] "Valid Inputs: "
      [,1]                                 [,2]                        
 [1,] "Afghanistan"                        "Cuba"                      
 [2,] "Albania"                            "Cyprus"                    
 [3,] "Algeria"                            "Czechia"                   
 [4,] "Andorra"                            "Côte d'Ivoire"             
 [5,] "Angola"                             "Denmark"                   
 [6,] "Antigua and Barbuda"                "Djibouti"                  
 [7,] "Argentina"                          "Dominica"                  
 [8,] "Armenia"                            "Dominican Republic"        
 [9,] "Australia"                          "Ecuador"                   
[10,] "Austria"                            "Egypt"                     
[11,] "Azerbaijan"                         "El Salvador"               
[12,] "Bahamas"                     

[1] "118.491555805603 km"

I just realized that it does not work very well for very high population densities due to the lack of datapoints for the creation of the function. Therefore, I will further submit an alternative code which I tried before switching to the method already presented. This function of pipe length will instead draw on an additional table containing data on the UN index of the percentage of urban population in a country.
1.) Here the base length the population of the region in question is multiplied with a factor representing the average pipe length per citizen. In this function the factor is derived from data for the city of Berlin, Germany, as an average size major city.
2.) Next a correction factor for urban regions is built, as the pipe length per citizen varies in urban and rural regions. A UN Index for the percentage of urban population in a country is used (drawing on a table derived from http://hdr.undp.org/en/data, dimension "Demography" selecting "Population, urban (%)"), when the density calculated from the input suggests that a rural region or a whole country is being evaluated. If the density is above 680 people per km², it is assumed that the entire investigated area is to be considered as urban.
urfac1 and urfac2 represent urban and rural pipe length factors. In rural regions the pipe length per citizen is assumed to be about five times higher than in rural regions (urfac2 <- 5). Each of these factors is multiplied with the respective percentages for urban (urper) and rural population in a country.
For solely urban areas (density higher than 680) the urper factor is 1 because the whole area is considered urban. Therefore urfac2 is zero so that no part of the population is calculated as rural.
3.) The third factor remains untouched by the changes.

In [19]:
pop_urban <- read.csv("Population_urban2.csv", header=TRUE, sep=",", dec=".", na.strings="..")

In [20]:
pipe_length2 <- function(m, n, o){
#1.)
  base_length <- n * 0.0027
    
#2.)
    if(o>=680){
  urfac1 <- 1
  urfac2 <- 0
  urper <- 1
} else {
  urfac1 <- 1
  urfac2 <- 5
  urper <- pop_urban[grep(m, pop_urban[,2]),3]/100
}
corr_urban <- base_length * urfac1 * urper + base_length * urfac2 * (1-urper)

  if (o>=680){
    access <- pop_water_urban[grep(m, pop_water_urban[,1]),2]/100
  } else {
    access <- pop_water[grep(m, pop_water[,2]),3]/100
  }
  final_length2 <- corr_urban * access
  return(final_length2)
}

In [21]:
input2 <- function(){
  print("Valid Inputs: ")
  print(regions)
  country2 <- readline (prompt="Enter country or region: ")
  population2 <- as.numeric(readline (prompt="Enter population: "))
  area2 <- as.numeric(readline(prompt="Enter area in km²: "))
  pop_density2 <- population2/area2
  final2 <- pipe_length2(country2, population2, pop_density2)
  paste0(final2, " km")
}

In [24]:
input2()

Enter country or region: Chile
Enter population: 5220161
Enter area in km²: 641.4
[1] "Valid Inputs: "
      [,1]                                 [,2]                        
 [1,] "Afghanistan"                        "Cuba"                      
 [2,] "Albania"                            "Cyprus"                    
 [3,] "Algeria"                            "Czechia"                   
 [4,] "Andorra"                            "Côte d'Ivoire"             
 [5,] "Angola"                             "Denmark"                   
 [6,] "Antigua and Barbuda"                "Djibouti"                  
 [7,] "Argentina"                          "Dominica"                  
 [8,] "Armenia"                            "Dominican Republic"        
 [9,] "Australia"                          "Ecuador"                   
[10,] "Austria"                            "Egypt"                     
[11,] "Azerbaijan"                         "El Salvador"               
[12,] "Bahamas"                  

[1] "14094.4347 km"